In [1]:
%load_ext autoreload
%autoreload 2

In [25]:
import sys
sys.path.append('../../src/generic')
import csv
import os
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_theme()
from sklearn.metrics import log_loss

from datasets import load_from_disk

In [5]:
from results.analyze_data import load_amazon_data

## Part 1 Test New Load Data Code

Note: I don't really have any demographic/attribute info for each user unfortunately

So the question of how to incorporate this auxillary information may be better answered with another dataset.

In [15]:
data_dir="/data/ddmg/redditlanguagemodeling/data/AmazonReviews/data"
data_path='amazon_v2.0/reviews.csv'
split_file_name="my_user_split.csv"
target_people_file="/data/ddmg/redditlanguagemodeling/results/amazon_reviews/clf/from_embeds/person_specific_my_split_n_500/selected_people.txt"
embeddings_path="/data/ddmg/redditlanguagemodeling/data/AmazonReviews/data/amazon_v2.0/embeddings/my_user_split_clf_post_embeddings/"
train_split_num=0
val_split_num=1
test_split_num=2

In [24]:
data_df = pd.read_csv(os.path.join(data_dir, data_path),
                      dtype={'reviewerID': str, 'asin': str, 'reviewTime': str, 'unixReviewTime': int,
                             'reviewText': str, 'summary': str, 'verified': bool, 'category': str,
                             'reviewYear': int},
                      keep_default_na=False, na_values=[], quoting=csv.QUOTE_NONNUMERIC)
data_df = data_df.rename(columns={"reviewerID": "user", "overall": "review_score"})
split_df = pd.read_csv(os.path.join(data_dir, 'amazon_v2.0', 'splits', split_file_name))
data_df["split"] = split_df["split"]
data_df = data_df[data_df["split"] != -1]
train_df = data_df[data_df["split"] == train_split_num]
val_df = data_df[data_df["split"] == val_split_num]
test_df = data_df[data_df["split"] == test_split_num]

In [40]:
train_df.head(5)

,user,asin,review_score,reviewTime,unixReviewTime,reviewText,summary,verified,category,reviewYear,split,text_embedding
34,A3HHQ7UIJJAOAV,B0009RF9DW,5.0,"06 4, 2013",1370304000,We both love the shower gel. It smells so goo...,shower gel used by grandmom and grand-daughter...,True,All_Beauty,2013,0.0,"[-0.010989412, -0.33095324, 0.12738927, -0.067..."
144,A173YMJ9XFVRSY,B0010ZBORW,5.0,"10 24, 2017",1508803200,Great for an at home spa experience. I have dr...,Moisturizing,False,All_Beauty,2017,0.0,"[-0.0020468049, -0.23871419, -0.08201841, -0.0..."
160,ALNFHVS3SC4FV,B0010ZBORW,5.0,"09 10, 2017",1505001600,Review for Loofah:\n\nI have tried loofahs in ...,Nice Loofah with Attached Suction Cup,False,All_Beauty,2017,0.0,"[0.027423665, -0.24965718, -0.005420671, -0.06..."
166,ANM9LSZS7C67V,B0010ZBORW,5.0,"09 8, 2017",1504828800,Good strong wooden handle that is easy to use....,Simple and precise,False,All_Beauty,2017,0.0,"[0.014582153, -0.3553315, -0.00086091936, -0.0..."
173,A3SMR4HRFJARSC,B0010ZBORW,4.0,"09 6, 2017",1504656000,Urban Spa Get a Grip Trio Manicure and Pedicur...,Works Well - Has Gripper Surface,False,All_Beauty,2017,0.0,"[-0.8540349, 0.97147715, 0.1957345, -0.0649095..."


In [39]:
# test add embeddings to df
train_df["text_embedding"] = list(train_embeds)

<ipython-input-39-0ab620ed14b0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["text_embedding"] = list(train_embeds)


In [19]:
# read embeddings from embeddings directory
train_embeds = np.load(os.path.join(embeddings_path, "train_my_user_split_split", "embeddings.npy"), allow_pickle=True)

In [30]:
test_embeds.shape

(108918, 768)

In [20]:
val_embeds = np.load(os.path.join(embeddings_path, "val_my_user_split_split", "embeddings.npy"), allow_pickle=True)
test_embeds = np.load(os.path.join(embeddings_path, "test_my_user_split_split", "embeddings.npy"), allow_pickle=True)

In [32]:
# embeddings from preprocessed dataset
# preprocessed path
preprocessed_path = "/data/ddmg/redditlanguagemodeling/data/AmazonReviews/data/amazon_v2.0/amazon_reviews_clf_processed_with_user_split_embeds"
train_embeds2 = load_from_disk(os.path.join(preprocessed_path, "train_my_user_split_split"))
val_embeds2 = load_from_disk(os.path.join(preprocessed_path, "val_my_user_split_split"))
test_embeds2 = load_from_disk(os.path.join(preprocessed_path, "test_my_user_split_split"))

In [33]:
len(train_embeds2["embeddings"])

323189

In [35]:
train_embeds3 = np.array(train_embeds2["embeddings"])

In [36]:
train_embeds3.shape

(323189, 768)

In [37]:
np.array_equal(train_embeds, train_embeds3)

True

In [31]:
len(test_df)

108918

In [42]:
data_df, train_df, val_df, test_df = load_amazon_data()

/data/ddmg/redditlanguagemodeling/reddit-personalized-lm/notebooks/AmazonReview_experiments/../../src/generic/results/analyze_data.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["text_embedding"] = list(train_embeds)
/data/ddmg/redditlanguagemodeling/reddit-personalized-lm/notebooks/AmazonReview_experiments/../../src/generic/results/analyze_data.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_df["text_embedding"] = list(val_embeds)
/data/ddmg/redditlanguagemodeling/reddit-personalize

In [43]:
len(data_df)

72709

In [44]:
len(train_df)

323189

In [45]:
len(val_df)

107395

In [46]:
len(test_df)

108918

## Part 2 Test New Get User Info Code

In [12]:
review_counts_by_user = data_df[["reviewerID", "overall"]].groupby(["reviewerID"]).count()

In [13]:
review_counts_by_user

,overall
reviewerID,
A101S5PLO0VRHQ,172
A10E0V7PGY34UZ,75
A10O7THJ2O20AG,75
A11P853U6FIKAM,278
A12O5K3EQ4MC7Z,249
...,...
AYT4FJYVCHYLE,75
AYVW3O6W8S5S4,75
AZD488SA9QMYF,75


In [ ]:
# check embeddings
